In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "../../data/SP500_EGS_Score_avarage_per_year.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
df.head(20)

In [ ]:
# all lower case

df.columns = map(str.lower, df.columns)

# - to _

df.columns = df.columns.str.replace("-", "_")

In [ ]:
len(df.company_symbol.unique())

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from helpers.text_preprocessing.preprocess_text import preprocess_text

In [ ]:
import pandas as pd

# Load or create your DataFrame (replace 'your_data.csv' with the path to your data file)
df = pd.read_csv(
    "../data/extracted_text_sustainability_reports.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
# Preprocess the text and get the preprocessed DataFrame
preprocessed_df = preprocess_text(df)

In [ ]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import plotly
from gensim import corpora
import mlflow

In [ ]:
import sys

sys.path.append("../")

In [ ]:
from models.LDA_optuna_tuning.tune_lda_optuna import train_lda, compute_coherence
from models.LDA_optuna_tuning.call_optuna_tune import (
    preprocess_data,
    execute_optuna_study,
)

In [ ]:
df = pd.read_csv(
    "../data/preprocessed_data_text_format.csv", index_col=0, parse_dates=True, sep=","
)

In [ ]:
df = df.head(200)

In [ ]:
# write the preprocessed data to csv
df.to_csv("../data/preprocessed_data_text_format_200.csv")

In [ ]:
preprocessed_stats = {
    "Number of Missing Values": df["preprocessed_content"].isna().sum(),
    "Number of Unique Texts": df["preprocessed_content"].nunique(),
    "Sample Texts": df["preprocessed_content"].sample(5).tolist(),
}

preprocessed_stats

In [ ]:
df

In [ ]:
if mlflow.active_run():
    mlflow.end_run()
execute_optuna_study(df, n_trials=200)

In [ ]:
import yfinance as yf

In [ ]:
# Create a ticker object
tesla = yf.Ticker("TSLA")

# To fetch row names from financials and put them in a list
financials_rows = tesla.financials.index.tolist()
print("Financials Rows:", financials_rows)

# To fetch row names from cashflow and put them in a list
cashflow_rows = tesla.cashflow.index.tolist()
print("Cashflow Rows:", cashflow_rows)

# To fetch row names from balance sheet and put them in a list
balance_rows = tesla.balance_sheet.index.tolist()
print("Balance Rows:", balance_rows)

print(tesla.info)

In [ ]:
import pandas as pd
from collections import Counter

df = pd.read_csv(
    "../data/ready_to_model/ready_to_model_df.csv",
    index_col=0,
    parse_dates=True,
    sep=",",
)

In [ ]:
# Filter out any non-string values in the 'preprocessed_content' column
df["preprocessed_content"] = df["preprocessed_content"].apply(
    lambda x: x if isinstance(x, str) else ""
)

# Tokenize the 'preprocessed_content' and flatten the list
all_words = [word for content in df["preprocessed_content"] for word in content.split()]

# Count the frequency of each word
counter = Counter(all_words)

# Display the 10 most common and 10 least common words
most_common_words = counter.most_common(300)
least_common_words = [word for word, count in counter.items() if count <= 2]


print("Most common words", most_common_words)
print("Least common words", least_common_words)
print(len(most_common_words))
print(len(least_common_words))

In [1]:
from sklearn.model_selection import train_test_split, KFold

import tpot2
import sklearn
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv")
df_sample = df.sample(100, random_state=100)

In [ ]:
df.head()

In [3]:
# columns to drop
columns_to_drop = [
    "e_score",
    "s_score",
    "g_score",
    "unnamed: 0",
    "filename",
    "ticker",
    "year",
    "preprocessed_content",
    "ner_entities",
    "company_symbol",
    "total_score",
]

# Separate features and target
y = df["total_score"]
X = df.drop(columns=columns_to_drop)

In [4]:
# drop the last two rows
X = X.iloc[:-2, :]

In [5]:
y = y.iloc[:-2]

In [6]:
X

,Unnamed: 0,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,balance_Accumulated Depreciation,balance_Gross PPE,balance_Other Properties,balance_Machinery Furniture Equipment,balance_Current Assets,balance_Other Current Assets,balance_Receivables,balance_Accounts Receivable,balance_Cash Cash Equivalents And Short Term Investments,balance_Cash And Cash Equivalents
0,0,0.001218,0.001870,0.002429,0.185920,0.003068,0.002446,0.002079,0.004457,0.002653,...,-2.413200e+09,5.228600e+09,7.463000e+08,2.622400e+09,5.330500e+09,478100000.0,2.978300e+09,2.261500e+09,1.472700e+09,1.472700e+09
1,1,0.000938,0.002150,0.001672,0.037319,0.000956,0.001582,0.000889,0.001315,0.001176,...,-9.779120e+08,2.139470e+09,4.126740e+08,8.163310e+08,NaN,NaN,7.948855e+09,6.701959e+09,NaN,2.521655e+10
2,2,0.001615,0.003504,0.003851,0.122123,0.012733,0.710308,0.002722,0.005965,0.081098,...,-5.416770e+08,9.470280e+08,9.470280e+08,3.826760e+08,6.635045e+09,108801000.0,3.810286e+09,3.810286e+09,1.728692e+09,1.728692e+09
3,3,0.000771,0.000802,0.000955,0.001158,0.000978,0.632143,0.000848,0.000936,0.000806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.001340,0.001617,0.001802,0.637962,0.001676,0.243458,0.001693,0.003728,0.001563,...,-2.117000e+08,8.993000e+08,1.560000e+07,2.030000e+08,3.600600e+09,65400000.0,8.077000e+08,8.077000e+08,2.551700e+09,1.319100e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,842,0.000242,0.000364,0.000372,0.000452,0.000292,0.000323,0.000341,0.000364,0.000253,...,-5.526380e+08,5.745470e+08,5.745470e+08,6.695770e+08,1.422425e+09,115436000.0,3.950710e+08,3.950710e+08,3.758800e+08,3.758800e+08
843,843,0.001257,0.001116,0.001339,0.002035,0.000916,0.001633,0.001734,0.001064,0.156478,...,-3.698000e+09,2.814000e+09,2.814000e+09,NaN,NaN,NaN,2.704000e+09,3.282000e+09,NaN,3.466000e+09
844,844,0.003077,0.339887,0.003480,0.050167,0.003389,0.015454,0.006424,0.004994,0.003117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.214700e+10
845,845,0.001088,0.775492,0.001835,0.053042,0.001516,0.002567,0.001855,0.001733,0.001633,...,-1.068800e+10,2.906200e+10,1.513000e+09,NaN,5.974000e+09,NaN,1.086000e+09,1.086000e+09,4.072000e+09,2.548000e+09


In [7]:
y

0      15.30
1      27.66
2      18.13
3      18.25
4      21.07
       ...  
842    28.98
843    18.73
844    21.70
845    30.41
846    19.73
Name: total_score, Length: 847, dtype: float64

In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=100
)

In [ ]:
scorer = sklearn.metrics.get_scorer("neg_mean_squared_error")

# Initialize TPOT2 regressor with K-Fold cross-validation
est = tpot2.TPOTEstimatorSteadyState(
    n_jobs=6,
    cv=KFold(n_splits=5),  # 5-Fold cross-validation
    verbose=2,
    classification=False,
    scorers=[scorer],
    scorers_weights=[1],
    max_eval_time_seconds=60 * 10,
    max_time_seconds=60 * 90,
)

# Fit the model
est.fit(X_train, y_train)
print('Done fitting/training TPOT2 session.')


df_individuals = est.evaluated_individuals

# Convert the 'mean_squared_error' column to numeric, errors='coerce' will replace non-numeric with NaN
df_individuals['mean_squared_error'] = pd.to_numeric(df_individuals['mean_squared_error'], errors='coerce')

# Drop NaN values
filtered_df = df_individuals.dropna(subset=['mean_squared_error'])

# Sort the DataFrame by 'mean_squared_error' and get the top 10
top_10_mse = filtered_df.nlargest(10, 'mean_squared_error')

print(est.pareto_front)
print(top_10_mse)
print(est.get_params())



In [ ]:
top_10_mse

In [ ]:
top_30_mse = filtered_df.nlargest(30, 'mean_squared_error')


In [ ]:
# save the top 30 models to csv
top_30_mse.to_csv("../data/model_data/top_30_mse.csv")

In [ ]:
top_30_mse

In [9]:
import sys

sys.path.append("../")

In [10]:
from models.XGBoost.train_and_evaluate_model import train_and_evaluate_model
from models.XGBoost.tune_xgb_hyperparameters import tune_xgb_hyperparameters


In [11]:
print(X_train.dtypes)


Unnamed: 0                                                    int64
topic_0                                                     float64
topic_1                                                     float64
topic_2                                                     float64
topic_3                                                     float64
                                                             ...   
balance_Other Current Assets                                float64
balance_Receivables                                         float64
balance_Accounts Receivable                                 float64
balance_Cash Cash Equivalents And Short Term Investments    float64
balance_Cash And Cash Equivalents                           float64
Length: 1326, dtype: object


In [ ]:
X_train = X_train.apply(pd.to_numeric, errors='ignore')


In [ ]:
best_params = tune_xgb_hyperparameters(X_train, y_train, n_trials=30)


In [ ]:
final_model, mse = train_and_evaluate_model(X_train, y_train, X_test, y_test, best_params)

In [ ]:
df

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_and_merge_data(df):
    # Create an empty list to store fetched data
    fetched_data = []
    
    for i, row in df.iterrows():
        ticker = row['ticker']
        year = row['year']
        
        # Initialize the Yahoo Finance object
        yf_ticker = yf.Ticker(ticker)
        
        # Fetch financials, cashflow, balance, and info
        financials = yf_ticker.financials
        cashflow = yf_ticker.cashflow
        balance = yf_ticker.balance_sheet
        info = yf_ticker.info  # This returns a dictionary
        
        # Filter only numerical values from info
        numerical_info = {k: v for k, v in info.items() if isinstance(v, (int, float))}
        
        # Filter data to match the specific year
        financials = financials.loc[:, pd.to_datetime(financials.columns).year == year]
        cashflow = cashflow.loc[:, pd.to_datetime(cashflow.columns).year == year]
        balance = balance.loc[:, pd.to_datetime(balance.columns).year == year]
        
        # Create a DataFrame from the numerical info
        info_df = pd.DataFrame([numerical_info])
        
        # Merge these new data points into a single DataFrame
        merged_data = pd.concat([financials, cashflow, balance, info_df], axis=1)
        
        # Add 'ticker' and 'year' columns
        merged_data['ticker'] = ticker
        merged_data['year'] = year
        
        # Append this merged data to the fetched_data list
        fetched_data.append(merged_data.reset_index(drop=True))
        
    # Concatenate all fetched data into a new DataFrame
    new_data = pd.concat(fetched_data, axis=0, ignore_index=True)
    
    # Finally, merge the new_data DataFrame with the original DataFrame
    final_df = pd.merge(df, new_data, on=['ticker', 'year'], how='left')
    
    return final_df


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import yfinance as yf
import pandas as pd


def fetch_data_for_row(named_tuple_row):
    ticker = "Unknown"  # Initialize with a default value
    year = "Unknown"    # Initialize with a default value
    
    try:
        ticker = named_tuple_row.ticker
        year = named_tuple_row.year

        yf_ticker = yf.Ticker(ticker)
        financials = yf_ticker.financials
        cashflow = yf_ticker.cashflow
        balance = yf_ticker.balance_sheet
        info = {k: v for k, v in yf_ticker.info.items() if isinstance(v, (int, float))}

        financials = financials.loc[:, pd.to_datetime(financials.columns).year == year].transpose()
        cashflow = cashflow.loc[:, pd.to_datetime(cashflow.columns).year == year].transpose()
        balance = balance.loc[:, pd.to_datetime(balance.columns).year == year].transpose()
        info_df = pd.DataFrame([info])

        financials.columns = 'financials_' + financials.columns.astype(str)
        cashflow.columns = 'cashflow_' + cashflow.columns.astype(str)
        balance.columns = 'balance_' + balance.columns.astype(str)
        info_df.columns = 'info_' + info_df.columns.astype(str)

        merged_data = pd.concat([financials, cashflow, balance, info_df], axis=1)
        merged_data['ticker'] = ticker
        merged_data['year'] = year

        return merged_data.reset_index(drop=True)
    except Exception as e:
        print(f"An error occurred in fetch_data_for_row for ticker: {ticker} and year: {year}. Error: {e}")
        return pd.DataFrame()



def fetch_and_merge_data(df):
    try:
        # Initialize an empty list to store fetched data
        fetched_data_list = []
        
        with ThreadPoolExecutor() as executor:
            fetched_data_list = list(executor.map(fetch_data_for_row, df.itertuples(index=False)))

        # Concatenate all the fetched data
        new_data = pd.concat([data.iloc[[0]] for data in fetched_data_list if not data.empty], ignore_index=True)

        # Debug: Print the shape and columns of new_data
        print(f"new_data shape: {new_data.shape}, columns: {new_data.columns}")

        # Merge new_data with df based on 'ticker' and 'year'
        final_df = pd.merge(df, new_data, on=['ticker', 'year'], how='left')

        return final_df
    except Exception as e:
        print(f"An error occurred in fetch_and_merge_data: {e}")
        return df  # Return the original DataFrame as a fallback




# Example usage
# df = pd.DataFrame({'ticker': ['AAPL', 'GOOGL'], 'year': [2020, 2021]})
# final_df = fetch_and_merge_data(df)


In [ ]:
df

In [ ]:
final_df = fetch_and_merge_data(df)

In [ ]:
df = final_df

In [ ]:
final_df.sample(10)

In [ ]:
# Calculate the percentage of missing values for each column
missing_percent = df_cleaned.isnull().mean() * 100

# Sort the columns by percentage of missing values in descending order
missing_percent_sorted = missing_percent.sort_values(ascending=False)

# Show the sorted series
print(missing_percent_sorted)


In [ ]:
nan_count = final_df.isna().sum()


In [ ]:
def remove_columns_with_nans(df, threshold=800):
    nan_count = final_df.isna().sum()
    columns_to_remove = nan_count[nan_count > threshold].index.tolist()
    df_cleaned = final_df.drop(columns=columns_to_remove)
    return df_cleaned

In [ ]:
df_cleaned = remove_columns_with_nans(df, threshold=300)


In [ ]:
df_cleaned



In [ ]:
df_cleaned.to_csv("../data/ready_to_model/df_cleaned_with_yfinance.csv")

In [ ]:
# Sorting columns by the number of NaN values (in descending order)
sorted_nan_count = nan_count.sort_values(ascending=False)

print(sorted_nan_count.value_counts())